#**PyUnloader**

Este es un programa que exporta los datos contenidos en una base de datos SYBASE SQL Anywhere version 7 y produce arvhivos con sentencias DDL en donde se crer tablas, vistas, se definen llaves primarias, indices y relaciones entre tablas.

Con este objetivo se realizan queries sobre las tablas del sistema :

**SYSUSERS** : contiene la informacion de los usuarios creados en la base de datos

**SYSTABLE** : contiene informacion de las tablas y vistas 

**SYSCOLUMNS** : contiene infromacion sobre las columnas de cada tabla

**SYSINDEXES** :  contiene infromacion sobre los indices y constraints creados para las tablas


In [441]:
import pyodbc  

DataSource = "DSN=BaseSQL32"
ExportFile = "PyUnload"
ExportDataDir = "C:\\PyUnload2\\PyUnload\\"
Usuario = 'PRG'
Filtro = ["Productos", "Gestores", "Usuarios", "Inventario", "Empresas"]
maxRows = 100

In [442]:
sql_user = ''' 
DROP USER "PRG";

GRANT CONNECT TO "PRG"   IDENTIFIED BY 
ENCRYPTED '\\x63\\x4A\\xC1\\xFB\\xC0\\xDD\\x71\\xE2\\xD4\\xD4\\x79\\x1F\\x43\\x3E\\xA1\\x82\\xF5\\x2C\\xFF\\xB7\\xB4\\xBF\\x57\\x0F\\x19\\x40\\xF6\\x09\\x13\\x6A\\x31\\x2F\\xE4\\x5A\\x93\\x22';

COMMENT ON USER "PRG" IS 'Usuario usado por los programas';

GRANT GROUP TO "PRG";

GRANT RESOURCE, DBA, REMOTE DBA TO "PRG";

'''
file = open(ExportFile +'-'+ Usuario +'.sql', "w")
file.write(sql_user)
file.close()

In [443]:
# Estrae la lista de TABLAS y VISTAS de un propietario especifico
 
def getSysTables(creator, tipoObjeto, filtro):

    lista_tablas = []
    creator_id = ""
    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()
    
    sql_usuarios  = "SELECT * FROM SYSUSERS WHERE NAME = " + "'" + creator + "'"
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()
    for row in rows:
        creator_id=row.suid

    if len(filtro)==0:
        sql_tablas = "SELECT * FROM SYSTABLE WHERE TABLE_TYPE = '" + tipoObjeto 
        sql_tablas = sql_tablas + "' AND CREATOR=" + str(creator_id)
        sql_tablas = sql_tablas + " ORDER BY TABLE_NAME "
    else:
        sql_tablas = "SELECT * FROM SYSTABLE WHERE TABLE_TYPE = '" + tipoObjeto 
        sql_tablas = sql_tablas + "' AND CREATOR=" + str(creator_id) 
        sql_tablas = sql_tablas + " AND ("
        n = 0
        for fil in filtro:
            sql_tablas = sql_tablas + " TABLE_NAME='" + fil +"'"
            if n<len(filtro)-1:
                sql_tablas = sql_tablas + " OR "
                n = n +1
        sql_tablas = sql_tablas + ") ORDER BY TABLE_NAME "

    cursor.execute(sql_tablas)
    rows = cursor.fetchall()
    
    print (sql_tablas)
    print(len(rows))

    for row in rows:
        tbl={}
        tbl["table_creator"] = creator
        tbl["table_id"] = row.table_id
        tbl["table_count"]= row.count
        tbl["table_name"]= row.table_name
        lista_tablas.append(tbl)

    cursor.close()
    conexionDB.close()

    return lista_tablas

In [444]:
# Se crea una lista que contiene la informacion principal de cada TABLA : Creador, Id y Nombre
#
#{'table_creator': 'PRG', 'table_id': 358, 'table_count': 0, 'table_name': 'ABC'}
#{'table_creator': 'PRG', 'table_id': 379, 'table_count': 4, 'table_name': 'ABCObsoletos'}
#{'table_creator': 'PRG', 'table_id': 380, 'table_count': 3, 'table_name': 'ABCUnidades'}
#...
#for usr in Usuarios:
lista_tablas = getSysTables(Usuario, 'BASE',Filtro)
file = open("Tablas-" + Usuario + ".txt", "a")
file.write('\n'.join(str(lt) for lt in lista_tablas))
file.close()

# Se crea un diccionario con la informacion principal de cada VISTA : Creador, Id y Nombre
#
#{'table_creator': 'PRG', 'table_id': 451, 'table_count': 0, 'table_name': 'ALFA1'}
#{'table_creator': 'PRG', 'table_id': 8899, 'table_count': 0, 'table_name': 'CajaDM1'}
#{'table_creator': 'PRG', 'table_id': 442, 'table_count': 0, 'table_name': 'ClienteFicha'}
#for usr in Usuarios:
lista_vistas = getSysTables(Usuario, 'VIEW', Filtro)
file = open("Vistas-" + Usuario + ".txt", "w")
file.write('\n'.join(str(lt) for lt in lista_tablas))
file.close()

SELECT * FROM SYSTABLE WHERE TABLE_TYPE = 'BASE' AND CREATOR=101 AND ( TABLE_NAME='Productos' OR  TABLE_NAME='Gestores' OR  TABLE_NAME='Usuarios' OR  TABLE_NAME='Inventario' OR  TABLE_NAME='Empresas') ORDER BY TABLE_NAME 
5
SELECT * FROM SYSTABLE WHERE TABLE_TYPE = 'VIEW' AND CREATOR=101 AND ( TABLE_NAME='Productos' OR  TABLE_NAME='Gestores' OR  TABLE_NAME='Usuarios' OR  TABLE_NAME='Inventario' OR  TABLE_NAME='Empresas') ORDER BY TABLE_NAME 
0


In [445]:
# Constuye una lista con la informacion de las columnas de la tabla indicada en el parametro

def getSysColumns(table_name):
    columnas = []
 
    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()

    sql_usuarios  = "SELECT * FROM SYSCOLUMNS WHERE TNAME = " + "'" + table_name + "'" + " ORDER BY COLNO"
    
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()
    for row in rows:
        obj = {}
        obj["col_name"]=row.cname
        obj["col_type"]=row.coltype
        obj["col_null"]=row.nulls
        obj["col_length"]=row.length
        obj["col_syslength"]=row.syslength
        obj["col_inPK"]=row.in_primary_key
        columnas.append(obj)

    cursor.close()
    conexionDB.close()

    return columnas

In [446]:
# Genera la sentencia DDL de CREATE TABLE de la tabla indicada en los parametros

def createTable(creator, table_name, columns):

    DDL= ""

    DDL = DDL +  "IF EXISTS (SELECT * FROM sysobjects WHERE name = \'" + table_name + "\' AND type = 'U') DROP TABLE " + creator + "." + table_name + ";\n"

    DDL = DDL + "CREATE TABLE " + "\"" + creator + "\".\"" + table_name +"\"\n" 
    DDL = DDL + "(" +"\n" 
    for col in columns:
        DDL = DDL + "\t" + col["col_name"] + " " + col["col_type"]

        if col["col_type"] != "long varchar" and col["col_type"] != "integer" and col["col_type"] != "decimal" and  col["col_type"] != "date":
            DDL = DDL + "(" + str(col["col_length"]) + ")" 
        if col["col_type"] == "decimal":
            DDL = DDL +  "(" + str(col["col_length"]) +","+ str(col["col_syslength"]) + ")" 
        
        if col["col_null"] == "N":
            DDL = DDL + " NOT NULL" +",\n"
        else:
            DDL = DDL + ",\n"

    DDL =  DDL + ")" + ";\n\n"
    #print(DDL)
    return DDL

In [447]:
# Escribe sentencias DDL de CREATE TABLE para las tablas encontradas

todas_columnas =[]

#for usr in Usuarios:
file = open(ExportFile +'-'+ Usuario +'.sql', "a")
for col in lista_tablas:
    columnas = getSysColumns(col["table_name"])
    sentenciaDDL = createTable(col["table_creator"], col["table_name"], columnas)
    file.write(sentenciaDDL)

    todas_columnas.append("\n" + col["table_name"])
    todas_columnas.append('\n'.join(str(x) for x in columnas))
file.close()

file = open("Columnas-" + Usuario +'.txt', "w")
file.write(('\n'.join(str(x) for x in todas_columnas)))
file.close()


In [448]:
# Extrae la lista de indices de la tabla indicada en los parametros

def getSysIndexes(table_name):

    indices = []
    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()

    sql_indices  = "SELECT * FROM SYSINDEXES WHERE INDEXTYPE = 'Non-unique' AND TNAME =" + "'" + table_name + "' order by iname"   

    cursor.execute(sql_indices)
    rows = cursor.fetchall()
    
    for row in rows:
        obj={}
        obj["creator"]=row.creator
        obj["table_name"]=row.tname
        obj["ind_name"]=row.iname
        colnames=row.colnames
        obj["columns"]=colnames.split(',')
        indices.append(obj)

    cursor.close()
    conexionDB.close()

    return indices

In [449]:
# Genera sentencia DDL de CREATE INDEX para la tabla indicada 

def createIndexes(indices):

    sentenciaCompleatDDL = "" 
    for ind in indices:
        sentenciaDDL = ""
        sentenciaDDL = "CREATE INDEX " + "\"" + ind["ind_name"] + "\" ON " + "\"" + ind["creator"] + "\".\"" + ind["table_name"] +"\"\n" 
        sentenciaDDL = sentenciaDDL + "(" +"\n" 
        for col in ind["columns"]:
            sentenciaDDL = sentenciaDDL + "\t"  + col + ",\n"
        sentenciaDDL = sentenciaDDL + ");" +"\n"
        sentenciaCompleatDDL = sentenciaCompleatDDL + sentenciaDDL
        
    return sentenciaCompleatDDL
    

In [450]:
# Escribe sentencia DDL de CREATE INDEX para todas las tablas encontradas

file = open(ExportFile +'-'+ Usuario +'.sql', "a")
for tbl in lista_tablas:
    indices_tabla =  getSysIndexes(tbl["table_name"])
    indicesDDL = createIndexes(indices_tabla)
    file.write(indicesDDL)

file.close()

In [461]:
def exportData(table_name, table_id, max_rows):

    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()
    
    if max_rows!=-1:
        sql  = "select top " + str(max_rows) + " * from " + table_name
    else:
        sql  = "select * from " + table_name

    cursor.execute(sql)
    rows = cursor.fetchall()
    archivo_dat =  ExportDataDir + str(table_id) + ".dat"
    
    if len(rows)>0:    
        file = open(archivo_dat, "w")
        for row in rows:
            r = str(row)
            r = r.replace('(','')
            r = r.replace(')','')
            r = r.replace('Decimal','')
            print(r)
            file.write(r+"\n")
        file.close()
    else:
        return archivo_dat, False

    return archivo_dat , True
    


In [467]:
def exportDataExample(table_name, table_id, max_rows):

    conexionDB = pyodbc.connect(DataSource)
    cursor = conexionDB.cursor()
    
    if max_rows!=-1:
        sql  = "select top " + str(max_rows) + " * from " + table_name
    else:
        sql  = "select * from " + table_name

    cursor.execute(sql)
    rows = cursor.fetchall()
    archivo_dat =  ExportDataDir + str(table_id) + ".dat"
    
    
    # build list of column names to use as dictionary keys from sql results
    columns = [column[0] for column in cursor.description]
    #print(columns)
    import json
    results = []

    for row in cursor.fetchall():
        results.append(dict(zip(columns, row)))
    
    output = {"MetaData": {}, "SRData": results}

    print(json.dumps(output, sort_keys=True, indent=4))

    return archivo_dat , True

In [469]:
archivos_dat = []
for col in lista_tablas:
    exportedFile, exportResult = exportDataExample(col["table_name"], col["table_id"],1)
    if exportResult:
        print (exportedFile)
        archivos_dat.append(col)
    break

['EmpresaId', 'Codigo', 'Nombre', 'Direccion1', 'Direccion2', 'Ciudad', 'Departamento', 'Telefono', 'Fax', 'NoRegistro', 'NIT', 'Giro', 'FechaInicial', 'FechaFinal', 'PIVA', 'EMail', 'MonedaId', 'PaisId', 'GerenteGeneral', 'Auditor', 'Contador', 'EstadoResultados', 'ValidaNIT', 'ValidaNEI', 'AdmiteConsumidorFinal', 'AdmiteClienteCero', 'FijarIVA', 'ModificarNombreTPOSCredito', 'PreciosConIVA', 'AproximarCentavosPlanilla', 'IntervClasifVentasContado', 'IntervClasifVentasCredito', 'IntervClasifPagosCredito', 'ContratosClasifPagoFinanciado', 'PrecioAplicadoId', 'RetenerRentaDeCR', 'MinimoEnMayoreo', 'UsarCostoPromEmpresa', 'EsGranContribuyente', 'ExigirDocIdentidad', 'ValidezCotizacion', 'TipoNumeracionPartidas', 'AbonosFINGeneranFC', 'NoEsAgentePercepcion', 'RetenerIVADeCR', 'ValidarNoRegistro', 'CorteUsaCorrelativoTK', 'DescAutNvosClientes', 'PorcDescAutNvosClientes', 'ProductosRequierenClase', 'NumPatronalISSS', 'AceptaDevolucionParcial', 'RepetirCodigoAlSuperarLineas', 'PrecioMayoreoA

In [462]:
def exportDataEmerson(table_name, table_id,max_rows):
    controlador = False
    copia_r = ''
    registro= ''
    autorizar = True

    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()
    
    if max_rows!=-1:
        sql  = "select top " + str(max_rows) + " * from " + table_name
    else:
        sql  = "select * from " + table_name

    cursor.execute(sql)
    rows = cursor.fetchall()
    archivo_dat =  ExportDataDir + str(table_id) + ".dat"
    
    if len(rows)>0:    
        file = open(archivo_dat, "w")
        for row in rows:
            r = str(row)
            r = r.replace('Decimal','')
            for z in r:
                if z == '(' or z == ')':
                    autorizar = False
                else: 
                    autorizar = True
                if copia_r == '(' and controlador == False:
                    controlador = True
                if autorizar == True:
                    if z == "'" and controlador == True:
                        pass
                    else:
                        registro +=z   
                copia_r = z
            file.write(registro+"\n")
            registro = ''    
        file.close()
    else:
        return archivo_dat, False
    
    return archivo_dat , True

In [463]:
# Exporta los datos de contenidos en las tablas 

archivos_dat = []
for col in lista_tablas:
    exportedFile, exportResult = exportData(col["table_name"], col["table_id"],1)
    if exportResult:
        print (exportedFile)
        archivos_dat.append(col)

1, '1', 'TECNO REPUESTOS, SA. DE CV.', '31 C. PTE. PASAJE 1 # 1635 COL. LAYCO', '', 'SAN SALVADOR', 'SAN SALVADOR', '2225-1060', '2226-9290', '42308-4', '0614-080984-001-9', 'COMPRA Y VENTA DE REPUESTOS AUTOMOTRICES', datetime.date2011, 12, 1, datetime.date2030, 12, 31, '13.00', 'info@tecnorepuestos.com', 1, 1, 'THANIA MAGAÑA', '', 'MARVIN MOLINA', '\\\\Server-tecno\\mtcorpor\\Contab32\\Resultecno.rpt', 'F', 'T', 'T', 'T', 'T', 'F', 'F', 'F', 360, 360, 360, 12, 3, 'F', '-0.01', 'F', 'F', 'F', 365, 'ANUAL', 'F', 'F', 'F', 'T', 'F', 'F', '0.00', 'F', '', 'T', 'F', 'F', 'F', '0.00', '0.00', '0.00', -1, '0.00'
C:\PyUnload2\PyUnload\177.dat
1, 1, 'FELIX GONZALEZ', 'F', ''
C:\PyUnload2\PyUnload\180.dat
1, 4, 2, 1, datetime.date1996, 4, 24, 'ING', '-1', 'INICIO INVENTARIO', 30, 0, 30, '8.020000', '0.000000', '8.020000', '0.270000', '0.270000', ' ', ' ', '0.267300', '', '', '', '', '', -1, -1
C:\PyUnload2\PyUnload\222.dat
6109, 1, 370, '1-31340-19197', '1-31340-19197', 117, '1-31340-19197', 'C

In [454]:
def insertData(dat, ExportDataDir):
   
    archivo_dat = str(dat["table_id"]) + '.dat'

    file = open(ExportFile +'-'+ Usuario +'.sql', "a")
    sentenciaInsert = "INPUT INTO " + dat["table_creator"] + "." + dat["table_name"] +"\n"
    sentenciaInsert = sentenciaInsert + "FROM \'" +  ExportDataDir + archivo_dat + "\'\n"  
    sentenciaInsert = sentenciaInsert +  "FORMAT ASCII\n"
    sentenciaInsert = sentenciaInsert +  "BY ORDER;\n\n"
    file.write(sentenciaInsert)
    file.close()

    return True 

In [455]:
for dat in archivos_dat:
    insertData(dat, ExportDataDir)